# Catboost Hyperparameter Sweep 20210918
Largely after https://www.kaggle.com/satorushibata/optimize-catboost-hyperparameter-with-optuna-gpu

# Setup

In [1]:
# two manual flags (ex-config)
colab = False
gpu_available = False

In [2]:
# basic imports
from pathlib import Path
import os
import math
from datetime import datetime
import random
import multiprocessing
import pickle

In [3]:
%matplotlib inline
%config Completer.use_jedi = False
os.environ['WANDB_NOTEBOOK_NAME'] = f"sweep_catboost_{datetime.now().strftime('%Y%m%d')}.ipynb"

In [4]:
# handle Google Colab-specific library installation/updating
if colab:
    # much of the below inspired by or cribbed from the May 2021 Kaggle Tabular Playground winner, at 
    # https://colab.research.google.com/gist/academicsuspect/0aac7bd6e506f5f70295bfc9a3dc2250/tabular-may-baseline.ipynb?authuser=1#scrollTo=LJoVKJb5wN0L
    
    # Kaggle API for downloading the datasets
    !pip install --upgrade -q kaggle

    # weights and biases
    !pip install -qqqU wandb
    
    # Optuna for parameter search
    !pip install -q optuna

    !pip install --upgrade xgboost

    # upgrade sklearn
    !pip install --upgrade scikit-learn

    !pip install category_encoders
    !pip install catboost

    # lighgbm gpu compatible
    !git clone --recursive https://github.com/Microsoft/LightGBM
    ! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
    
    # this part is from https://github.com/rapidsai/gputreeshap/issues/24
    !pip install cmake --upgrade
    # !pip install sklearn --upgrade
    !git clone --recursive https://github.com/dmlc/xgboost
    %cd /content/xgboost
    !mkdir build
    %cd build
    !cmake .. -DUSE_CUDA=ON
    !make -j4
    %cd /content/xgboost/python-package
    !python setup.py install --use-cuda --use-nccl
    !/opt/bin/nvidia-smi
    !pip install shap
    

Now, non-stdlib imports

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import log_loss, roc_auc_score
import wandb
from optuna.integration.wandb import WeightsAndBiasesCallback
# from wandb.xgboost import wandb_callback
# from wandb.lightgbm import wandb_callback
# from sklearn.impute import KNNImputer, StandardImputer
# import timm

import seaborn as sns

from catboost import CatBoostClassifier
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MaxAbsScaler, StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures
# from sklearn.feature_selection import SelectKBest, f_regression
from joblib import dump, load
# feature engineering tools
# from sklearn.feature_selection import mutual_info_regression
# import featuretools as ft


from optuna.samplers import TPESampler
import optuna
import catboost
from sklearn.utils import resample
import sklearn.metrics

Now, datapath setup

In [6]:
# # This is the code for reading the train.csv and converting it to a .feather file
# df = pd.read_csv(datapath/'train.csv', index_col='id', low_memory=False)
# df.index.name = None
# df.to_feather(path='./dataset_df.feather')

In [7]:
if colab:
    # mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # handling datapath
    datapath = Path('/content/drive/MyDrive/kaggle/tabular_playgrounds/sep2021/')
    
else:
    # if on local machine
    datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/')    
    


In [8]:

n_trials = int(1000)
SEED = 42

In [9]:
# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
seed_everything(SEED)

## Ex-Model Config

In [10]:
# meta-config for preprocessing and cross-validation, but NOT for model parameters
# in the sweep version, this includes both ex-model parameters and defaults for model parameters
exmodel_config = {
    # model config
    "library": 'catboost',
#     "model": XGBClassifier,
#     "n_estimators": 100, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "test_size": 0.2,
#     "reg_lambda": None, 
    "scaler": "sklearn.preprocessing.StandardScaler()", # TODO: experiment with others (but imputation may be slow)
    "scale_b4_impute": False,
    "imputer": "sklearn.impute.SimpleImputer(strategy='median', add_indicator=True)",
    "knn_imputer_n_neighbors": None, # None if a different imputer is used
#     "feature_selector": SelectKBest,
#     "k_best": 80,
#     "feature_selection_scoring": f_regression,
    'random_state': SEED,
    'optuna': True,
    'optuna_trials': 1000,
#     'subsample': 1,
#     'cross_val_strategy': None, # None for holdout, or the relevant sklearn class
#     'kfolds': 1, # if 1, that means just doing holdout
#     'test_size': 0.2,
    # these are XGBoost default (my choice) params 
#     "tree_method": "auto", # set to 'gpu_hist' to try GPU if available
#     "booster": 'gbtree', # dart may be marginally better, but will opt for this quicker approach as a default
#     "n_estimators": 200, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "n_jobs": -1,
#     "verbosity": 1,
#     "subsample": 1,
#     'features_created': False,
#     'feature_creator': None,
}

wandb_kwargs = {
    # wandb config
    'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
    'project': '202109_Kaggle_tabular_playground',
    'tags': ['sweep'],
    'notes': "Sweep for CatBoost using Optuna",
    'config': exmodel_config,
}

## Data Setup

**TODO** Write some conditional logic here to automate it -- possibly as part of a sklearn.*pipeline

In [11]:
X_source = 'X_NaNcounts_SummaryStats_imputed-Median-wIndicators-StandardScaled.feather'
X_train = pd.read_feather(datapath/X_source) 
y_train = load(datapath/'y.joblib')    
# X.index.name = 'id'
# y.index.name = 'id'
X = np.array(X_train)
y = np.array(y_train)

del X_train, y_train

In [12]:
exmodel_config['feature_count'] = X.shape[1]
exmodel_config['feature_generator'] = "Summary statistics"
exmodel_config['X_source'] = X_source

# Experiment setup

In [13]:
# wandb_kwargs = {
#     # wandb config:
#     'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
#     'project': '202109_Kaggle_tabular_playground',
#     'tags': ['sweep'],
#     'notes': "Sweep for CatBoost using Optuna",
#     'config': exmodel_config,
# }

In [14]:
# originally from https://www.kaggle.com/satorushibata/optimize-catboost-hyperparameter-with-optuna-gpu
def objective(trial):
    # split the (original Kaggle training) data into partitions
    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=int(SEED), shuffle=True)
    # create wrappers for the training and validation partitions
    train_pool = catboost.Pool(train_x, train_y)
    test_pool = catboost.Pool(test_x, test_y)
    
    # experimental parameters
    params = {
        'iterations' : trial.suggest_int('iterations', 50, 4000),                         
        'depth' : trial.suggest_int('depth', 3, 10),                                       
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 0.3),               
        'random_strength' :trial.suggest_int('random_strength', 0, 100), 
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 2, 70),
        'border_count': trial.suggest_int('border_count', 50, 275),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
#         'learning_rate' :trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
#         'max_leaves': trial.suggest_int('max_leaves', 32, 128)
    }
    # instantiate the model, with some parameters locked in, and experimnental ones passed via splat 
    model = CatBoostClassifier(
        loss_function="Logloss",
        eval_metric="AUC",
#         task_type="GPU",
#         l2_leaf_reg=50,
        random_seed=SEED,
        verbose=False,
#         border_count=64,
        **params
    )        
#     wandb.init(wandb_kwargs)
    # fit the model on the training set -- note the use of the catboost.Pool instance
    model.fit(train_pool)
    # generate predictions
    preds = model.predict(test_pool)
    # rounds to the nearest integer, and the nearest even in case of _.5s
    pred_labels = np.rint(preds)
    # calls sklearn.utils.resample... not entirely sure why
    y_pred_boot = resample(pred_labels, n_samples = len(train_y))
    # Evaluation
    ROC_AUC_Score = roc_auc_score(train_y, y_pred_boot)
    print('ROC AUC Score of CatBoost =', ROC_AUC_Score)
    wandb.log({'valid_AUC': ROC_AUC_Score,
               'catboost_params': str(model.get_all_params())
              })
#     wandb.finish()
    return ROC_AUC_Score

In [15]:
wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)

<ipython-input-15-69ea9289a2cf>:1: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find sweep_catboost_20210919.ipynb
wandb: Currently logged in as: hushifang (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [16]:
study = optuna.create_study(direction = "maximize", sampler = TPESampler(seed=int(SEED)), study_name='CatBoost_20210919')
# setting n_jobs=1 bc wandb apparently requires this
study.optimize(objective, n_trials = n_trials, n_jobs=1, callbacks = [wandbc]) #n_jobs = multiprocessing.cpu_count())

[I 2021-09-19 15:37:54,499] A new study created in memory with name: CatBoost_20210919
[I 2021-09-19 15:45:22,247] Trial 0 finished with value: 0.5005467813689807 and parameters: {'iterations': 1529, 'depth': 10, 'learning_rate': 0.06504856968981275, 'random_strength': 60, 'l2_leaf_reg': 12, 'border_count': 85, 'bagging_temperature': 0.017073967431528128, 'od_type': 'IncToDec'}. Best is trial 0 with value: 0.5005467813689807.
wandb: WARNING Step must only increase in log calls.  Step 0 < 1; dropping {'iterations': 1529, 'depth': 10, 'learning_rate': 0.06504856968981275, 'random_strength': 60, 'l2_leaf_reg': 12, 'border_count': 85, 'bagging_temperature': 0.017073967431528128, 'od_type': 'IncToDec', 'value': 0.5005467813689807}.


ROC AUC Score of CatBoost = 0.5005467813689807


[I 2021-09-19 15:48:18,656] Trial 1 finished with value: 0.5009843470941473 and parameters: {'iterations': 2847, 'depth': 3, 'learning_rate': 0.2526878207508456, 'random_strength': 84, 'l2_leaf_reg': 16, 'border_count': 91, 'bagging_temperature': 0.05415244119402541, 'od_type': 'Iter'}. Best is trial 1 with value: 0.5009843470941473.
wandb: WARNING Step must only increase in log calls.  Step 1 < 2; dropping {'iterations': 2847, 'depth': 3, 'learning_rate': 0.2526878207508456, 'random_strength': 84, 'l2_leaf_reg': 16, 'border_count': 91, 'bagging_temperature': 0.05415244119402541, 'od_type': 'Iter', 'value': 0.5009843470941473}.


ROC AUC Score of CatBoost = 0.5009843470941473


[I 2021-09-19 15:50:52,882] Trial 2 finished with value: 0.4993992183124174 and parameters: {'iterations': 1756, 'depth': 5, 'learning_rate': 0.032781876533976156, 'random_strength': 14, 'l2_leaf_reg': 22, 'border_count': 132, 'bagging_temperature': 0.6672367170464207, 'od_type': 'IncToDec'}. Best is trial 1 with value: 0.5009843470941473.
wandb: WARNING Step must only increase in log calls.  Step 2 < 3; dropping {'iterations': 1756, 'depth': 5, 'learning_rate': 0.032781876533976156, 'random_strength': 14, 'l2_leaf_reg': 22, 'border_count': 132, 'bagging_temperature': 0.6672367170464207, 'od_type': 'IncToDec', 'value': 0.4993992183124174}.


...

[I 2021-09-19 20:58:22,117] Trial 82 finished with value: 0.4999553647216982 and parameters: {'iterations': 2560, 'depth': 4, 'learning_rate': 0.003091010618453093, 'random_strength': 29, 'l2_leaf_reg': 12, 'border_count': 245, 'bagging_temperature': 26.92759807073714, 'od_type': 'IncToDec'}. Best is trial 55 with value: 0.5013894302775861.
wandb: WARNING Step must only increase in log calls.  Step 82 < 83; dropping {'iterations': 2560, 'depth': 4, 'learning_rate': 0.003091010618453093, 'random_strength': 29, 'l2_leaf_reg': 12, 'border_count': 245, 'bagging_temperature': 26.92759807073714, 'od_type': 'IncToDec', 'value': 0.4999553647216982}.


ROC AUC Score of CatBoost = 0.4999553647216982


[I 2021-09-19 21:01:31,198] Trial 83 finished with value: 0.49982323173077936 and parameters: {'iterations': 3177, 'depth': 3, 'learning_rate': 0.006430042753972595, 'random_strength': 70, 'l2_leaf_reg': 2, 'border_count': 223, 'bagging_temperature': 0.03533243750492781, 'od_type': 'IncToDec'}. Best is trial 55 with value: 0.5013894302775861.
wandb: WARNING Step must only increase in log calls.  Step 83 < 84; dropping {'iterations': 3177, 'depth': 3, 'learning_rate': 0.006430042753972595, 'random_strength': 70, 'l2_leaf_reg': 2, 'border_count': 223, 'bagging_temperature': 0.03533243750492781, 'od_type': 'IncToDec', 'value': 0.49982323173077936}.


ROC AUC Score of CatBoost = 0.49982323173077936


KeyboardInterrupt: 

In [17]:
dump(study.best_trial.params, filename='optuna_catboost_best_20210919.joblib')
# pickle.dump(study.best_trial.params, open('CatBoost_Hyperparameter.pickle', 'wb'))
# print('CatBoost Hyperparameter:', study.best_trial.params)

['optuna_catboost_best_20210919.joblib']

In [18]:
study.best_trial.params

{'iterations': 3493,
 'depth': 5,
 'learning_rate': 0.09397459954141321,
 'random_strength': 43,
 'l2_leaf_reg': 26,
 'border_count': 239,
 'bagging_temperature': 12.532400413798356,
 'od_type': 'Iter'}